In [1]:
import numpy as np
import pandas as pd
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from keras.optimizers import Adam


Using TensorFlow backend.


In [2]:
def build_discriminator():
    model = Sequential()
    model.add(Dense(41, input_dim=41, activation='relu'))  # discriminator takes 41 values from our dataset
    model.add(Dense(30, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # outputs 0 to 1, 1 being read and 0 being fake

    # model.summary()

    attack = Input(shape=(41,))
    validity = model(attack)

    return Model(attack, validity)

In [3]:
def build_generator(hidden1, hidden2, hidden3):
    model = Sequential()
    model.add(Dense(hidden1, input_dim=41))  # arbitrarily selected 100 for our input noise vector?
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(hidden2))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(hidden3))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(41, activation='relu'))  # outputs a generated vector of the same size as our data (41)

    # model.summary()

    noise = Input(shape=(41,))
    attack = model(noise)
    return Model(noise, attack)

In [4]:
def trainGAN(gen_hidden1, gen_hidden2, gen_hidden3):
    batch_size = 256
    epochs = 7000
    optimizer = Adam(0.0002, 0.5)
    
    dataframe = pd.read_csv('smurfOnly.csv').sample(500) # sample 100 data points randomly from the csv
    
    # apply "le.fit_transform" to every column (usually only works on 1 column)
    le = LabelEncoder()
    dataframe_encoded = dataframe.apply(le.fit_transform)
    dataset = dataframe_encoded.values
    
    #to visually judge results
    print("Real smurf attacks:")
    print(dataset[:2])
    
    # Set X as our input data and Y as our label
    X_train = dataset[:, 0:41].astype(float)
    Y_train = dataset[:, 41]
    
    # labels for data. 1 for valid attacks, 0 for fake (generated) attacks
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    # build the discriminator portion
    discriminator = build_discriminator();
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    # build the generator portion
    generator = build_generator(gen_hidden1, gen_hidden2, gen_hidden3)
    
    #input and output of our combined model
    z = Input(shape=(41,))
    attack = generator(z)
    validity = discriminator(attack)
    
    # build combined model from generator and discriminator
    combined = Model(z, validity)
    combined.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    #break condition for training (when diverging)
    loss_increase_count = 0;
    prev_g_loss = 0;
    
    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------
        
        # selecting batch_size random attacks from our training data
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        attacks = X_train[idx]
        
        # generate a matrix of noise vectors
        noise = np.random.normal(0, 1, (batch_size, 41))
        
        # create an array of generated attacks
        gen_attacks = generator.predict(noise)
        
        # loss functions, based on what metrics we specify at model compile time
        d_loss_real = discriminator.train_on_batch(attacks, valid)
        d_loss_fake = discriminator.train_on_batch(gen_attacks, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # generator loss function
        g_loss = combined.train_on_batch(noise, valid)
        
        if epoch % 100 == 0:
            print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f] [Loss change: %.3f, Loss increases: %.0f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss, g_loss - prev_g_loss, loss_increase_count))
        
        # if our generator loss icreased this iteration, increment the counter by 1
        if (g_loss - prev_g_loss) > 0:
            loss_increase_count = loss_increase_count + 1
        else: 
            loss_increase_count = 0  # otherwise, reset it to 0, we are still training effectively
            
        prev_g_loss = g_loss
            
        if loss_increase_count > 5:
            print('Stoping on iteration: ', epoch)
            break
            
        if epoch % 20 == 0:
            f = open("GANresultsSmurf.txt", "a")
            np.savetxt("GANresultsSmurf.txt", gen_attacks, fmt="%.0f")
            f.close()
            
    # peek at our results
    results = np.loadtxt("GANresultsSmurf.txt")
    print("Generated Smurf attacks: ")
    print(results[:2])
        
        


In [5]:
# Initialize Random Number Generator
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

# load dataset

dataframe = pd.read_csv("smurfAndNormal.csv")#, header=True) 

# samples 10000 random data points from 500k
dataframe = dataframe.sample(n=15000)
# LabelEncoder, turns all our categorical data into integers
le = LabelEncoder()

# apply "le.fit_transform" to every column (usually only works on 1 column)
dataframe_encoded = dataframe.apply(le.fit_transform)
attack_labels = le.classes_
indices_of_smurf = np.where(attack_labels == 'smurf.')
smurf_index = indices_of_smurf[0]
dataset = dataframe_encoded.values

print(attack_labels)
print(smurf_index)

#Set X as our input data and Y as our label
X = dataset[:,0:41].astype(float)
Y = dataset[:,41]


['normal.' 'smurf.']
[1]


In [6]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
# print(dummy_y)
#print(len(dummy_y[0]))
num_of_classes = len(dummy_y[0])  # the length of dummy y is the number of classes we have in our small sample
# since we are randomly sampling from a large dataset, we might not get 1 of every class in our sample
# we need to set output layer to be equal to the length of our dummy_y vectors
print(num_of_classes)


2


In [7]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    
    inputs = 41
    hidden_layer1 = 10
    hidden_layer2 = 5
    hidden_layer3 = 0
    outputs = num_of_classes  #needs to be this variable in case we forget to sample. Could end up having 10 classes or 12, etc
    
    model.add(Dense(hidden_layer1, input_dim=inputs, activation='relu'))
    if hidden_layer2 != 0:
        model.add(Dense(hidden_layer2, activation='relu'))
    if hidden_layer3 != 0:
        model.add(Dense(hidden_layer3, activation='relu'))
    model.add(Dense(outputs, activation='softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) #optimizer=adam
    return model

In [8]:
#for i in range(0,10):
estimator = KerasClassifier(build_fn=baseline_model, epochs=32, batch_size=200, verbose=2)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)
y_pred = cross_val_predict(estimator, X, dummy_y, cv=kfold)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)

trained_classifier = estimator.fit(X, Y)
print(type(estimator))

cm = confusion_matrix(Y, y_pred)
print(cm)
print("total: " + str(cm.sum()))
print("accuracy: " + str(np.trace(cm) / cm.sum()))
print("Matthews correlation coefficient: " + str(matthews_corrcoef(Y, y_pred)))



print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

f = open("discriminatorResults.txt", "a+")
f.write("TP: %d, FP: %d, FN: %d, TN: %d\n" % (cm[0][0], cm[0][1], cm[1][0], cm[1][1]))
f.close()


Epoch 1/32
 - 0s - loss: 1.2883 - acc: 0.8981
Epoch 2/32
 - 0s - loss: 0.3875 - acc: 0.9749
Epoch 3/32
 - 0s - loss: 0.2963 - acc: 0.9804
Epoch 4/32
 - 0s - loss: 0.0531 - acc: 0.9954
Epoch 5/32
 - 0s - loss: 0.0046 - acc: 0.9996
Epoch 6/32
 - 0s - loss: 0.0036 - acc: 0.9996
Epoch 7/32
 - 0s - loss: 0.0031 - acc: 0.9995
Epoch 8/32
 - 0s - loss: 0.0023 - acc: 0.9997
Epoch 9/32
 - 0s - loss: 0.0018 - acc: 0.9997
Epoch 10/32
 - 0s - loss: 0.0017 - acc: 0.9998
Epoch 11/32
 - 0s - loss: 0.0018 - acc: 0.9998
Epoch 12/32
 - 0s - loss: 0.0013 - acc: 0.9999
Epoch 13/32
 - 0s - loss: 0.0018 - acc: 0.9997
Epoch 14/32
 - 0s - loss: 0.0014 - acc: 0.9999
Epoch 15/32
 - 0s - loss: 0.0015 - acc: 0.9999
Epoch 16/32
 - 0s - loss: 0.0016 - acc: 0.9998
Epoch 17/32
 - 0s - loss: 0.0011 - acc: 0.9999
Epoch 18/32
 - 0s - loss: 0.0017 - acc: 0.9999
Epoch 19/32
 - 0s - loss: 0.0014 - acc: 0.9999
Epoch 20/32
 - 0s - loss: 0.0014 - acc: 0.9999
Epoch 21/32
 - 0s - loss: 0.0013 - acc: 0.9999
Epoch 22/32
 - 0s - lo

 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 15/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 16/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 17/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 18/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 19/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 20/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 21/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 22/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 23/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 24/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 25/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 26/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 27/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 28/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 29/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 30/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 31/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 32/32
 - 0s - loss: 11.9167 - acc: 0.2607
Epoch 1/32
 - 0s - loss: 12.4745 - acc: 0.2199
Epoch 2/32
 - 0s - loss: 11.9274 - acc: 0.2600
Epoch 

Epoch 27/32
 - 0s - loss: 0.0024 - acc: 0.9999
Epoch 28/32
 - 0s - loss: 0.0024 - acc: 0.9999
Epoch 29/32
 - 0s - loss: 0.0024 - acc: 0.9999
Epoch 30/32
 - 0s - loss: 0.0024 - acc: 0.9999
Epoch 31/32
 - 0s - loss: 0.0024 - acc: 0.9999
Epoch 32/32
 - 0s - loss: 0.0024 - acc: 0.9999
Epoch 1/32
 - 0s - loss: 11.9221 - acc: 0.2603
Epoch 2/32
 - 0s - loss: 11.9216 - acc: 0.2604
Epoch 3/32
 - 0s - loss: 11.9216 - acc: 0.2604
Epoch 4/32
 - 0s - loss: 11.9215 - acc: 0.2604
Epoch 5/32
 - 0s - loss: 11.9215 - acc: 0.2604
Epoch 6/32
 - 0s - loss: 11.9215 - acc: 0.2604
Epoch 7/32
 - 0s - loss: 11.9215 - acc: 0.2604
Epoch 8/32
 - 0s - loss: 11.9215 - acc: 0.2604
Epoch 9/32
 - 0s - loss: 11.9215 - acc: 0.2604
Epoch 10/32
 - 0s - loss: 11.9215 - acc: 0.2604
Epoch 11/32
 - 0s - loss: 11.9215 - acc: 0.2604
Epoch 12/32
 - 0s - loss: 11.9215 - acc: 0.2604
Epoch 13/32
 - 0s - loss: 11.9215 - acc: 0.2604
Epoch 14/32
 - 0s - loss: 11.9215 - acc: 0.2604
Epoch 15/32
 - 0s - loss: 11.9215 - acc: 0.2604
Epoch 1

Epoch 10/32
 - 0s - loss: 0.0081 - acc: 0.9986
Epoch 11/32
 - 0s - loss: 0.0081 - acc: 0.9990
Epoch 12/32
 - 0s - loss: 0.0075 - acc: 0.9988
Epoch 13/32
 - 0s - loss: 0.0072 - acc: 0.9989
Epoch 14/32
 - 0s - loss: 0.0067 - acc: 0.9990
Epoch 15/32
 - 0s - loss: 0.0059 - acc: 0.9990
Epoch 16/32
 - 0s - loss: 0.0054 - acc: 0.9990
Epoch 17/32
 - 0s - loss: 0.0048 - acc: 0.9990
Epoch 18/32
 - 0s - loss: 0.0045 - acc: 0.9991
Epoch 19/32
 - 0s - loss: 0.0039 - acc: 0.9995
Epoch 20/32
 - 0s - loss: 0.0038 - acc: 0.9995
Epoch 21/32
 - 0s - loss: 0.0032 - acc: 0.9995
Epoch 22/32
 - 0s - loss: 0.0027 - acc: 0.9995
Epoch 23/32
 - 0s - loss: 0.0025 - acc: 0.9996
Epoch 24/32
 - 0s - loss: 0.0022 - acc: 0.9996
Epoch 25/32
 - 0s - loss: 0.0022 - acc: 0.9997
Epoch 26/32
 - 0s - loss: 0.0019 - acc: 0.9996
Epoch 27/32
 - 0s - loss: 0.0021 - acc: 0.9996
Epoch 28/32
 - 0s - loss: 0.0022 - acc: 0.9996
Epoch 29/32
 - 0s - loss: 0.0023 - acc: 0.9997
Epoch 30/32
 - 0s - loss: 0.0025 - acc: 0.9996
Epoch 31/32
 

In [ ]:
"""
f = open("GeneratorHypersAbove50percentAccuracy.txt", "w")
f.write("""""" Hidden layer counts for Generator model that resulted in over 50% generated attacks labeled correctly:
    ------------------------------------------------------------------------------------------------
    """""")
f.close()
"""

while(1):
    # generate random numbers for the hidden layer sizes of our generator
    gen_hidden1 = np.random.randint(1, 101)
    gen_hidden2 = np.random.randint(1, 101)
    gen_hidden3 = np.random.randint(1, 101)
    
    i = 0
    
    
    # train 5 times on each setup, in case we get unlucky initalization on an otherwise good setup
    while i < 5:
        # create a unique filename in case we want to store the results (good accuracy)
        result_filename = "GANresultsSmurf%.0f%.0f%.0fiter%.0f.txt" % (gen_hidden1, gen_hidden2, gen_hidden3, i)

        trainGAN(gen_hidden1, gen_hidden2, gen_hidden3)
        
        # load generate attacks from file
        results = np.loadtxt("GANresultsSmurf.txt")

        # predict attack lables (as encoded integers)
        y_pred = estimator.predict(results)
        print(y_pred)

        # create appropriate labels for our generated smurf attacks
        smurf_labels = np.full((len(results),), smurf_index[0])

        # convert integer labels back to string, get all unique strings and their count
        predicted_as_label = attack_labels[y_pred]
        unique_labels = np.unique(predicted_as_label)

        for label in unique_labels:
            print("Attack type: %s     number predicted:  %.0f" % (label, len(np.where(predicted_as_label == label)[0])))
    
        print()
        # create a confusion matrix of the results
        cm = confusion_matrix(smurf_labels, y_pred)
        
        accuracy = np.trace(cm) / cm.sum()
        print(cm)
        print("total: " + str(cm.sum()))
        print("accuracy: " + str(accuracy))
        
        if accuracy > .50:
            f = open("GeneratorHypersAbove50percentAccuracySmurf.txt", "a")
            f.write("""
            
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accuracy: %.3f
Generator hidden layer 1 size: %.0f
Generator hidden layer 2 size: %.0f
Generator hidden layer 3 size: %.0f
Iteration %.0f
Result file name: %s
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~""" % (accuracy, gen_hidden1, gen_hidden2, gen_hidden3, i, result_filename))
            f.close()
            
            f = open(result_filename, "w")
            f.close()
            np.savetxt(result_filename, results, fmt="%.0f")
        
        i = i + 1
            


Real smurf attacks:
[[ 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 34 34
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 34 34
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
0 [D loss: 1.369914, acc.: 49.41%] [G loss: 0.832706] [Loss change: 0.833, Loss increases: 0]
100 [D loss: 0.481357, acc.: 53.32%] [G loss: 0.507623] [Loss change: 0.002, Loss increases: 0]
200 [D loss: 0.698590, acc.: 50.98%] [G loss: 0.346337] [Loss change: 0.003, Loss increases: 1]
300 [D loss: 0.805833, acc.: 51.95%] [G loss: 0.301139] [Loss change: -0.019, Loss increases: 1]
400 [D loss: 0.807378, acc.: 52.34%] [G loss: 0.283261] [Loss change: 0.003, Loss increases: 1]
500 [D loss: 0.868982, acc.: 50.78%] [G loss: 0.265436] [Loss change: -0.009, Loss increases: 0]
600 [D loss: 0.851985, acc.: 52.34%] [G loss: 0.272886] [Loss change: 0.006, Loss increases: 1]
700 [D loss: 0.889143, acc.: 51.17%] [G loss:

0 [D loss: 2.000813, acc.: 5.66%] [G loss: 0.655410] [Loss change: 0.655, Loss increases: 0]
100 [D loss: 0.680956, acc.: 50.00%] [G loss: 0.342085] [Loss change: 0.002, Loss increases: 0]
200 [D loss: 0.870636, acc.: 50.00%] [G loss: 0.243498] [Loss change: 0.004, Loss increases: 1]
300 [D loss: 0.903877, acc.: 50.00%] [G loss: 0.215737] [Loss change: -0.006, Loss increases: 0]
400 [D loss: 1.025675, acc.: 50.00%] [G loss: 0.189370] [Loss change: -0.006, Loss increases: 1]
500 [D loss: 1.082617, acc.: 50.00%] [G loss: 0.168070] [Loss change: -0.006, Loss increases: 0]
600 [D loss: 1.105909, acc.: 50.00%] [G loss: 0.162318] [Loss change: 0.001, Loss increases: 0]
700 [D loss: 1.115358, acc.: 50.20%] [G loss: 0.154664] [Loss change: 0.007, Loss increases: 0]
800 [D loss: 1.133312, acc.: 49.61%] [G loss: 0.145472] [Loss change: -0.008, Loss increases: 1]
900 [D loss: 1.075602, acc.: 50.59%] [G loss: 0.146471] [Loss change: -0.000, Loss increases: 0]
Stoping on iteration:  984
Generated S

1800 [D loss: 0.746961, acc.: 50.00%] [G loss: 0.299782] [Loss change: -0.009, Loss increases: 2]
1900 [D loss: 0.740326, acc.: 50.00%] [G loss: 0.318409] [Loss change: -0.007, Loss increases: 2]
2000 [D loss: 0.703898, acc.: 50.00%] [G loss: 0.331572] [Loss change: -0.001, Loss increases: 0]
2100 [D loss: 0.682882, acc.: 49.41%] [G loss: 0.353279] [Loss change: 0.010, Loss increases: 0]
2200 [D loss: 0.671015, acc.: 50.00%] [G loss: 0.362897] [Loss change: -0.005, Loss increases: 1]
2300 [D loss: 0.686701, acc.: 49.02%] [G loss: 0.372910] [Loss change: -0.012, Loss increases: 1]
2400 [D loss: 0.664711, acc.: 49.22%] [G loss: 0.400424] [Loss change: 0.008, Loss increases: 1]
2500 [D loss: 0.629667, acc.: 49.22%] [G loss: 0.414556] [Loss change: 0.003, Loss increases: 0]
2600 [D loss: 0.595640, acc.: 50.00%] [G loss: 0.433688] [Loss change: 0.009, Loss increases: 0]
2700 [D loss: 0.604124, acc.: 49.41%] [G loss: 0.446339] [Loss change: -0.010, Loss increases: 1]
2800 [D loss: 0.575466, 

Real smurf attacks:
[[ 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 29 29
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 29 29
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]]
0 [D loss: 0.412177, acc.: 50.00%] [G loss: 0.355540] [Loss change: 0.356, Loss increases: 0]
100 [D loss: 1.024772, acc.: 50.00%] [G loss: 0.175924] [Loss change: -0.007, Loss increases: 1]
200 [D loss: 1.018396, acc.: 50.00%] [G loss: 0.170692] [Loss change: -0.003, Loss increases: 0]
300 [D loss: 1.071105, acc.: 50.00%] [G loss: 0.165448] [Loss change: 0.001, Loss increases: 0]
400 [D loss: 1.052665, acc.: 49.80%] [G loss: 0.166338] [Loss change: -0.000, Loss increases: 0]
500 [D loss: 1.038666, acc.: 49.61%] [G loss: 0.170035] [Loss change: 0.001, Loss increases: 0]
600 [D loss: 1.055449, acc.: 49.22%] [G loss: 0.166851] [Loss change: -0.004, Loss increases: 0]
700 [D loss: 1.020982, acc.: 50.00%] [G los

1100 [D loss: 0.966074, acc.: 51.95%] [G loss: 0.285716] [Loss change: -0.026, Loss increases: 1]
1200 [D loss: 0.989661, acc.: 52.15%] [G loss: 0.274708] [Loss change: -0.014, Loss increases: 2]
1300 [D loss: 0.973209, acc.: 51.76%] [G loss: 0.258805] [Loss change: -0.011, Loss increases: 1]
1400 [D loss: 0.964913, acc.: 50.20%] [G loss: 0.270557] [Loss change: 0.004, Loss increases: 2]
1500 [D loss: 0.992721, acc.: 50.78%] [G loss: 0.277195] [Loss change: 0.011, Loss increases: 1]
1600 [D loss: 1.054746, acc.: 50.59%] [G loss: 0.257157] [Loss change: -0.003, Loss increases: 0]
1700 [D loss: 1.071178, acc.: 50.20%] [G loss: 0.257933] [Loss change: 0.022, Loss increases: 0]
1800 [D loss: 1.076211, acc.: 51.95%] [G loss: 0.236343] [Loss change: -0.013, Loss increases: 1]
1900 [D loss: 1.117893, acc.: 49.61%] [G loss: 0.224511] [Loss change: -0.016, Loss increases: 1]
2000 [D loss: 1.144363, acc.: 50.59%] [G loss: 0.227577] [Loss change: -0.003, Loss increases: 0]
2100 [D loss: 1.123589,

2400 [D loss: 0.856407, acc.: 49.22%] [G loss: 0.307188] [Loss change: 0.001, Loss increases: 0]
2500 [D loss: 0.868314, acc.: 48.63%] [G loss: 0.315089] [Loss change: 0.006, Loss increases: 0]
2600 [D loss: 0.870165, acc.: 48.05%] [G loss: 0.323533] [Loss change: -0.005, Loss increases: 0]
2700 [D loss: 0.861470, acc.: 49.41%] [G loss: 0.341017] [Loss change: -0.005, Loss increases: 3]
2800 [D loss: 0.863443, acc.: 49.02%] [G loss: 0.348392] [Loss change: -0.010, Loss increases: 2]
2900 [D loss: 0.832983, acc.: 49.02%] [G loss: 0.366706] [Loss change: 0.011, Loss increases: 0]
3000 [D loss: 0.791599, acc.: 49.80%] [G loss: 0.402550] [Loss change: 0.032, Loss increases: 0]
3100 [D loss: 0.752916, acc.: 49.41%] [G loss: 0.412651] [Loss change: -0.001, Loss increases: 1]
3200 [D loss: 0.774370, acc.: 49.02%] [G loss: 0.430347] [Loss change: 0.029, Loss increases: 0]
3300 [D loss: 0.846478, acc.: 48.05%] [G loss: 0.433283] [Loss change: -0.001, Loss increases: 1]
3400 [D loss: 0.829615, a

2800 [D loss: 1.145083, acc.: 49.02%] [G loss: 0.228794] [Loss change: -0.008, Loss increases: 0]
2900 [D loss: 1.175910, acc.: 50.59%] [G loss: 0.249706] [Loss change: 0.004, Loss increases: 2]
3000 [D loss: 1.170147, acc.: 51.17%] [G loss: 0.247164] [Loss change: 0.008, Loss increases: 2]
3100 [D loss: 1.136567, acc.: 50.78%] [G loss: 0.244557] [Loss change: -0.001, Loss increases: 0]
3200 [D loss: 1.150687, acc.: 49.61%] [G loss: 0.261465] [Loss change: 0.024, Loss increases: 0]
3300 [D loss: 1.193667, acc.: 51.95%] [G loss: 0.249195] [Loss change: 0.001, Loss increases: 0]
3400 [D loss: 1.177250, acc.: 51.95%] [G loss: 0.251563] [Loss change: -0.016, Loss increases: 1]
3500 [D loss: 1.234760, acc.: 50.39%] [G loss: 0.259953] [Loss change: 0.003, Loss increases: 0]
3600 [D loss: 1.207679, acc.: 50.00%] [G loss: 0.244876] [Loss change: -0.013, Loss increases: 2]
3700 [D loss: 1.179514, acc.: 52.73%] [G loss: 0.257993] [Loss change: -0.000, Loss increases: 1]
3800 [D loss: 1.155360, a

3900 [D loss: 1.271728, acc.: 50.20%] [G loss: 0.149580] [Loss change: 0.004, Loss increases: 0]
4000 [D loss: 1.237411, acc.: 50.59%] [G loss: 0.162733] [Loss change: 0.012, Loss increases: 0]
4100 [D loss: 1.256129, acc.: 50.59%] [G loss: 0.161857] [Loss change: 0.002, Loss increases: 0]
4200 [D loss: 1.254623, acc.: 50.20%] [G loss: 0.159809] [Loss change: 0.002, Loss increases: 1]
4300 [D loss: 1.163949, acc.: 50.20%] [G loss: 0.204209] [Loss change: 0.001, Loss increases: 0]
4400 [D loss: 1.240158, acc.: 50.00%] [G loss: 0.183845] [Loss change: -0.002, Loss increases: 4]
4500 [D loss: 1.229719, acc.: 50.39%] [G loss: 0.167192] [Loss change: -0.006, Loss increases: 0]
4600 [D loss: 1.213512, acc.: 51.17%] [G loss: 0.182493] [Loss change: 0.002, Loss increases: 0]
4700 [D loss: 1.179009, acc.: 51.37%] [G loss: 0.175984] [Loss change: -0.005, Loss increases: 2]
Stoping on iteration:  4738
Generated Smurf attacks: 
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0. 

6200 [D loss: 1.079033, acc.: 50.39%] [G loss: 0.218363] [Loss change: -0.001, Loss increases: 0]
6300 [D loss: 1.052607, acc.: 50.39%] [G loss: 0.228840] [Loss change: 0.005, Loss increases: 1]
6400 [D loss: 1.034466, acc.: 50.59%] [G loss: 0.230449] [Loss change: 0.010, Loss increases: 0]
6500 [D loss: 1.027664, acc.: 50.00%] [G loss: 0.227739] [Loss change: 0.002, Loss increases: 0]
6600 [D loss: 1.002580, acc.: 50.00%] [G loss: 0.234540] [Loss change: -0.001, Loss increases: 0]
6700 [D loss: 0.994606, acc.: 50.00%] [G loss: 0.236368] [Loss change: 0.005, Loss increases: 0]
6800 [D loss: 1.012600, acc.: 50.00%] [G loss: 0.238135] [Loss change: -0.003, Loss increases: 0]
6900 [D loss: 0.956698, acc.: 50.00%] [G loss: 0.256798] [Loss change: -0.003, Loss increases: 0]
Generated Smurf attacks: 
[[ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  3.  0. 57. 57.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  2.  